<h3>Задание 1. Вы провели эксперимент c упрощением формы заказа в магазине Утконос и получили результаты по метрике конверсий в покупку. Выберите метод оценки и оцените, есть ли статистически значимые различия между конверсиями в двух группах при alpha = 5%. Дайте краткие рекомендации команде.</h3>

Результаты эксперимента:<br>
 1) Число юзеров в группах, которые заходили на сайт в период эксперимента: n1 = 15550 и n2 = 15550.<br>
 2) Число юзеров в группах, которые совершили хотя бы одну покупку за период эксперимента: n1 = 164 и n2 = 228.<br>
 3) Конверсии: conv1 = 1.05%, conv2 = 1.47%.<br>

Метод статистического анализа:<br>
 * Хи-квадрат.<br>

Результаты анализа:<br>
<br>
    ![Sample 2 is more successful](scrnsht1.png "Результат")

Рекомендации:<br>
 1) Рекомендуется распространить упрощенную форму заказа на всех пользователей магазина.<br>

<h3>Задание 1. Сравниваем метрику конверсия в покупку. Размер выборки - 10000 элементов в каждой группе. Какой статистический критерий тут лучше всего подойдёт и почему?</h3>

Метрика "конверсия в покупку" является отношением числа пользователей, оформивших заказ, к числу всех пользователей, т.е. данная метрика является долей. Статистический анализ при сравнении долей в двух группах предполагает преимущественно использование двух методов - Хи-квадрат и z-критерий долей. При размере выборки 10000 применима центральная предельная теорема. Исследуемые в задаче выборки независимы (рандомизированы). Предполагаем, что математическое ожидание в группах не будет одинаковым. В таком случаем для сравнении групп следует прибегнуть к F-критерию Фишера. При равенстве же математических ожиданий возможны следующие варианты: стандартное отклонение в группах равно сигме генеральной совокупности и известно - используем z-критерий, либо если не известно - используем t-критерий Стьюдента. Если стандартные отклонений в группах не равны и известны - используем z-критерий, и если не известны - t-критерий Стьюдента с поправкой Аспина-Уэлча. Почему так? Понятия не имею!